In [18]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document


# Charger les guides depuis un fichier JSON
def load_guides(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        guides = json.load(f)
    return guides


# Charger les posts depuis un fichier JSON
def load_posts(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        posts = json.load(f)
    return posts


# Convertir les posts et guides en vecteurs et créer un retriever LangChain
def index_data_embeddings(
    posts, guides, model_name="sentence-transformers/all-MiniLM-L6-v2"
):
    # Construire les textes et les objets Document pour les posts
    documents = []
    for p in posts:
        text_comments = ""
        for comment in p["comments"]:
            text_comments += comment + "\n"
        documents.append(
            Document(
                page_content=f"{p['titre']} - {p['contenu']}",
                metadata={
                    "comments": text_comments,
                    "url": p["url"],
                    "titre": p["titre"],
                    "contenu": p["contenu"],
                },
            )
        )

    # Construire les textes et les objets Document pour les guides
    for g in guides:
        documents.append(
            Document(
                page_content=f"{g['dataType']} - {g['type']} {g['subject']} : {g['title']} {(g['url'])}",
                metadata={
                    "dataType": g["dataType"],
                    "type": g["type"],
                    "subject": g["subject"],
                    "title": g["title"],
                    "category": g["category"],
                    "summary": g["summary"],
                    "url": g["url"],
                    "guideid": g["guideid"],
                },
            )
        )

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)

    # Créer des embeddings avec LangChain
    embedding_model = HuggingFaceEmbeddings(model_name=model_name)
    vector_store = FAISS.from_documents(splits, embedding_model)

    return vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"k": 2, "score_threshold": 0.3},
    )

In [19]:
posts = load_posts("./data/techsupport_posts.json")

guides = load_guides("./data/guides.json")

retriever = index_data_embeddings(posts, guides)

In [20]:
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_KEY")

In [21]:
import ollama
from langchain_core.language_models import LLM
from typing import List


class OllamaLLM(LLM):
    model: str = "mistral"

    def _call(self, prompt: str, stop: List[str] = None) -> str:
        response = ollama.chat(
            model=self.model, messages=[{"role": "user", "content": prompt}]
        )
        return response["message"]["content"]

    @property
    def _identifying_params(self) -> dict:
        return {"model": self.model}

    @property
    def _llm_type(self) -> str:
        return "ollama"

In [22]:
from langchain.prompts import PromptTemplate

# Le prompt pour le modèle
final_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
L'utilisateur pose la question suivante :

➡️ {question}

Tu disposes uniquement des documents suivants : des guides techniques et des posts Reddit pertinents.
Ces contenus incluent des descriptions générales, des conseils pratiques, des solutions proposées par la communauté, et parfois des instructions techniques détaillées.

🎯 Ta mission :

    Base strictement ta réponse sur les informations présentes dans les documents fournis ci-dessous ({context}).

    N'utilise aucune connaissance extérieure. Si une information n’est pas présente, indique-le explicitement.

    Fournis une réponse structurée, professionnelle et en français.

📚 Sources disponibles :

{context}

🛠 Format de réponse attendu :

🔍 Analyse du problème :

[Présente une synthèse du problème posé, uniquement en te basant sur les documents.]

✅ Vérifications préalables recommandées :

[Liste les éléments à inspecter ou tester avant toute manipulation, tels que suggérés dans les documents.]

📝 Procédure détaillée proposée :

[Structure la procédure étape par étape : “Étape 1”, “Étape 2”… en t’appuyant sur les guides ou les conseils Reddit.]

💡 Conseils ou précautions à prendre :

[Ajoute ici uniquement les recommandations explicitement mentionnées dans les documents.]

🔗 Sources consultées :

[Liste les URL des documents (guides ou posts Reddit) utilisés pour construire la réponse, selon les métadonnées disponibles.]

📌 Important :
Tu dois strictement t'appuyer sur les contenus fournis dans {context}.
Aucune inférence ou ajout personnel n’est autorisé. Si la réponse n’est pas déductible des documents, indique-le clairement.
""",
)

In [23]:
import requests


def get_guide_steps(guideid):
    url = f"https://www.ifixit.com/api/2.0/guides/{guideid}"
    response = requests.get(url)

    if response.status_code != 200:
        return {
            "error": f"Échec de récupération du guide {guideid}, code: {response.status_code}"
        }

    data = response.json()
    steps = []

    cpt_steps = 0

    for step in data.get("steps", []):
        cpt_steps += 1
        step_texts = [
            line["text_rendered"]
            for line in step.get("lines", [])
            if "text_rendered" in line
        ]
        steps.append({"stepno": cpt_steps, "text": step_texts})

    return steps

In [24]:
def format_documents(docs):
    formatted_docs = []

    for doc in docs:
        guide_id = doc.metadata.get("guideid")
        if guide_id:
            guide_steps = get_guide_steps(guide_id)
            print(len(guide_steps))
            guide_infos = ""
            for guide in guide_steps:
                step_text = "\n".join(guide["text"])
                guide_infos += "\n" + f"Step {guide['stepno']}:\n" + step_text
            if guide_infos not in doc.page_content:
                doc.page_content += guide_infos

        metadata_text = "\n".join(
            f"{key}: {value}" for key, value in doc.metadata.items()
        )

        formatted_doc = f"""---\n📄 **Contenu** :\n{doc.page_content}\n\n🔖 **Métadonnées** :\n{metadata_text}\n"""
        formatted_docs.append(formatted_doc)

    print("\n\n".join(formatted_docs))
    return "\n\n".join(formatted_docs)

In [25]:
from langchain_openai import ChatOpenAI

# llm = OllamaLLM()

llm = ChatOpenAI(openai_api_key=OPENAI_KEY, model="gpt-4.1", temperature=0.0)

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


# Chaîne RAG
def create_rag_chain(retriever):
    return (
        {
            "context": retriever | format_documents,
            "question": RunnablePassthrough(),
        }
        | final_prompt_template
        | llm
        | StrOutputParser()
    )

# Multi Query

In [27]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Ta tâche est de générer cinq reformulations différentes de la question posée par l’utilisateur afin de retrouver des documents pertinents dans une base de données vectorielle.
En proposant plusieurs perspectives sur la question, ton objectif est d’aider l’utilisateur à surmonter certaines limites de la recherche par similarité basée sur la distance.
Fournis uniquement ces questions alternatives, en ajoutant la question originale traduite, chacune séparée par un saut de ligne.
Répond en **Anglais**
Question initiale : {question}
""",
)

In [28]:
from json import dumps, loads


def get_unique_union(documents: list[list[Document]]) -> list[Document]:
    """Renvoie une liste de documents uniques à partir d'une liste de listes de documents."""
    # Aplatir la liste
    flattened_docs = [
        dumps(doc.__dict__, sort_keys=True) for sublist in documents for doc in sublist
    ]
    # Supprimer les doublons
    unique_docs = list(set(flattened_docs))
    # Reconvertir en objets Document
    return [Document(**loads(doc)) for doc in unique_docs]

In [29]:
generate_queries = prompt_template | llm | StrOutputParser() | (lambda x: x.split("\n"))

In [30]:
retrieval_chain = generate_queries | retriever.map() | get_unique_union

In [31]:
question = "Pourquoi mon PC ne démarre pas ?"
queries = generate_queries.invoke(question)
print("🔍 Queries générées :", queries)

🔍 Queries générées : ["Why won't my computer turn on?", '', 'What could be causing my PC to fail to start up?', '', "What are the possible reasons my computer isn't powering up?", '', "How can I troubleshoot a PC that doesn't boot?", '', "What should I do if my computer won't start?"]


In [32]:
question = "Pourquoi mon PC ne démarre pas ?"
docs = retrieval_chain.invoke(question)
print("🔍 Nombre de documents récupérés :", len(docs))

c:\Users\Coco\Desktop\TB\venv\Lib\site-packages\langchain_core\vectorstores\base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='a0c516a6-940c-49ba-844f-3a10f7e77f63', metadata={'comments': 'Reddit image posts can not contain additional text besides the title. And text is crucial to explain a problem. So the decision is between text only or picture only. And since putting a picture link into text is much easier than putting text in a picture, text posts it is.\n\nYes, it\'s not ideal as it is, but it\'s a Reddit limitation rather than a subreddit policy limitation.\n[deleted]\nopen windows snipping tool, take screenshot, ctrl+c then go to imgur, click new post and paste it there\nYou can link as many images in the body of your text explanation as you like. There is no rule against this.\n\nIf you are asking to make image posts instead of text posts, that\'s a bad idea. Then you would have to give a textual description in the comments instead of up top

🔍 Nombre de documents récupérés : 7


In [38]:
rag_chain = create_rag_chain(retrieval_chain)
response = rag_chain.invoke(question)
print(response)

c:\Users\Coco\Desktop\TB\venv\Lib\site-packages\langchain_core\vectorstores\base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='a0c516a6-940c-49ba-844f-3a10f7e77f63', metadata={'comments': 'Reddit image posts can not contain additional text besides the title. And text is crucial to explain a problem. So the decision is between text only or picture only. And since putting a picture link into text is much easier than putting text in a picture, text posts it is.\n\nYes, it\'s not ideal as it is, but it\'s a Reddit limitation rather than a subreddit policy limitation.\n[deleted]\nopen windows snipping tool, take screenshot, ctrl+c then go to imgur, click new post and paste it there\nYou can link as many images in the body of your text explanation as you like. There is no rule against this.\n\nIf you are asking to make image posts instead of text posts, that\'s a bad idea. Then you would have to give a textual description in the comments instead of up top

---
📄 **Contenu** :
After this, all was fine until, my next technical issue.

&#x200B;

What's happening is now when I press the power button sometimes my computer sometimes turns on and sometimes doesn't.

&#x200B;

When it doesn't turn on the power button lights up as if it was turning on, so does the power supply, and looking inside it seems everything is moving and working it just doesn't turn on my keyboard, mouse and doesn't display anything on my screen.

&#x200B;

When it doesn't turn on it takes a bit of faffing around such as turning the computer back off again by the power button, turning it on again, turning it off, switching the plugs off, switching plugs on, and then turning the computer on. Even sometimes I have to turn everything off, open the computer up and make sure everything is connected correctly and it will work, although when opening the computer everything seems connected well enough.

&#x200B;

🔖 **Métadonnées** :
comments: It sounds like they did not put the 